# Import the required libraries

In [1]:
import pandas as pd

# Import the required jason file as a dataframe

In [2]:
df = pd.read_json("Cell_Phones_and_Accessories.json")

# Lets check the shape

In [19]:
df.shape

(760450, 12)

# Lets check the first five rows of data

In [3]:
df.head()

Rating  Rev_verify  Review_Date          IC             Prod_meta  \
0       5        True   09 1, 2015  B009XD5TPQ                  None   
1       5        True   01 9, 2016  B016MF3P3K                  None   
2       5        True  04 21, 2013  B008DC8N5G                  None   
3       3        True  02 27, 2013  B0089CH3TM  {'Color:': ' Green'}   
4       4        True  12 19, 2013  B00AKZWGAC                  None   

    Reviewer_Name                                             Review  \
0  Sunny  Zoeller  Bought it for my husband. He's very happy with it   
1   Denise Lesley  Great screen protector.  Doesn't even seem as ...   
2            Emir  Saved me lots of money! it's not gorilla glass...   
3           Alyse  The material and fit is very nice, but the col...   
4         TechGuy  This last me about 3 days till i have to charg...   

                                Rev_summ  Review_timestamp Useful Prod_img  
0                   He's very happy with        1441065600   None     None  
1                             Five Stars        1452297600   None     None  
2  As long as you know how to put it on!        1366502400   None     None  
3                      Good case overall        1361923200      3     None  
4                        Awesome Battery        1387411200   None     None

# Lets create a new feature having Review and Review summary combined

In [5]:
df['final_text'] = df['Review'] + df['Rev_summ']

# Select only useful columns

In [6]:
dff = df[['IC','Rating','final_text']]

# Lets check the first five rows of data

In [7]:
dff.head()

IC  Rating                                         final_text
0  B009XD5TPQ       5  Bought it for my husband. He's very happy with...
1  B016MF3P3K       5  Great screen protector.  Doesn't even seem as ...
2  B008DC8N5G       5  Saved me lots of money! it's not gorilla glass...
3  B0089CH3TM       3  The material and fit is very nice, but the col...
4  B00AKZWGAC       4  This last me about 3 days till i have to charg...

# Lets check how many unique items are there

In [10]:
dff['IC'].nunique()

48134

# Lets check for missing values

In [12]:
dff.isnull().sum()

IC              0
Rating          0
final_text    848
dtype: int64

# Lets drop the rows having missing values

In [16]:
dff.dropna()

IC  Rating                                         final_text
0       B009XD5TPQ       5  Bought it for my husband. He's very happy with...
1       B016MF3P3K       5  Great screen protector.  Doesn't even seem as ...
2       B008DC8N5G       5  Saved me lots of money! it's not gorilla glass...
3       B0089CH3TM       3  The material and fit is very nice, but the col...
4       B00AKZWGAC       4  This last me about 3 days till i have to charg...
...            ...     ...                                                ...
760445  B00C3V9M8A       4                                Very goodFour Stars
760446  B0178BYS24       5  My name is Cynthia Beard and I believe that th...
760447  B009KY47CE       4  This iphone case is very durable and long last...
760448  B00X60AYDY       5                                    greatFive Stars
760449  B00QLD3YTO       5  So so, it did not look like the picture advert...

[759602 rows x 3 columns]

# Lets confirm the missing values

In [17]:
dff.isnull().sum()

IC            0
Rating        0
final_text    0
dtype: int64

# Lets check the shape of data

In [18]:
dff.shape

(759602, 3)

# Lets get more info about data

In [20]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 759602 entries, 0 to 760449
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   IC          759602 non-null  object
 1   Rating      759602 non-null  int64 
 2   final_text  759602 non-null  object
dtypes: int64(1), object(2)
memory usage: 23.2+ MB


# Import more required libraries

In [21]:
import string
import nltk
from collections import Counter
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

# Lets create functions for data cleaning

In [22]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [23]:
# Let us create a function for cleaning the text data
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [24]:
#Lets test our custom function
mytest = 'hi my name is 123@, why are @#$ you running here and eating'

In [25]:
clean_text(mytest)

'hi name run eat'

In [26]:
# clean text data
dff["final_text"] = dff["final_text"].apply(lambda x: clean_text(x))

C:\Anaconda3\envs\dlib_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
#Lets check our data frame
dff.head()

IC  Rating                                         final_text
0  B009XD5TPQ       5             bought husband he's happy ithe's happy
1  B016MF3P3K       5  great screen protector even seem though theref...
2  B008DC8N5G       5  save lot money gorilla glass careful subject e...
3  B0089CH3TM       3  material fit nice color neon green expect woul...
4  B00AKZWGAC       4  last day till charge take forever charge make ...

In [70]:
#Lets get the list of  unique values of  Item codes

In [38]:
x =list(set(dff['IC'].values))

In [40]:
len(x)

48133

In [71]:
#Lets create empty list to store  corresponding data

In [60]:
ID = []
MAX_RATING = []
MIN_RATING = []
AVG_RATING = []



In [72]:
#Lets analyse the max,min and average rating for each item code

In [61]:
for i in x:
    ID.append(i)
    MAX_RATING.append(dff[dff['IC'] == i]['Rating'].max())
    MIN_RATING.append(dff[dff['IC'] == i]['Rating'].min())
    AVG_RATING.append(dff[dff['IC'] == i]['Rating'].mean())

In [73]:
#Lets create a dictionary for above analysed data

In [64]:
dic= {'ID' : ID, 'MAX_RATING': MAX_RATING,'MIN_RATING':MIN_RATING,'AVG_RATING':AVG_RATING}

In [74]:
#Lets create a dataframe from above analysis 

In [66]:
dataframe = pd.DataFrame(dic)

In [68]:
dataframe.head()

ID  MAX_RATING  MIN_RATING  AVG_RATING
0  B00MEYUM1W           5           1    4.300000
1  B015J1K0PO           5           1    3.117647
2  B011KR1Z64           5           1    3.416667
3  B01G8E96C4           5           1    3.916667
4  B013ZC4Z8C           5           4    4.750000

In [75]:
#save the dataframe to a json file
dataframe.to_json('review.json')

In [77]:
#Lets check our json file

In [76]:
dx = pd.read_json('review.json')
dx.head()

ID  MAX_RATING  MIN_RATING  AVG_RATING
0  B00MEYUM1W           5           1    4.300000
1  B015J1K0PO           5           1    3.117647
2  B011KR1Z64           5           1    3.416667
3  B01G8E96C4           5           1    3.916667
4  B013ZC4Z8C           5           4    4.750000